# OPEN WEATHER ONE CALL API FOR WEATHER VARIABLES
---
---

In [1]:
# https://openweathermap.org/api/one-call-3
# Lesson 5.01 - APIs
# https://note.nkmk.me/en/python-unix-time-datetime/

# Imports
---

In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests

import datetime

%matplotlib inline
pd.set_option('display.max_columns', None)

# Testing: Historical Daily Weather from One Call API
---

In [3]:
url_hist_daily = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658362065&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_daily_req = requests.get(url_hist_daily)
print(f'Status: {hist_daily_req.status_code}')

wd_hist = hist_daily_req.json()
print(wd_hist.keys())

wd_hist_df = json_normalize(wd_hist['data'])
wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_hist_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'data'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,2022-07-20 17:07:45,313.66,311.77,1016,15,282.02,0,0.45,272


In [4]:
wd_hist

{'lat': 38.7735,
 'lon': -121.3637,
 'timezone': 'America/Los_Angeles',
 'timezone_offset': -25200,
 'data': [{'dt': 1658362065,
   'sunrise': 1658321791,
   'sunset': 1658374005,
   'temp': 313.66,
   'feels_like': 311.77,
   'pressure': 1016,
   'humidity': 15,
   'dew_point': 282.02,
   'clouds': 0,
   'wind_speed': 0.45,
   'wind_deg': 272,
   'wind_gust': 0.45,
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}]}]}

In [5]:
url_hist_point = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658363457&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_point_req = requests.get(url_hist_point)
print(f'Status: {hist_point_req.status_code}')

wd_hist = hist_point_req.json()
print(wd_hist.keys())

wd_hist_df = json_normalize(wd_hist['data'])
wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_hist_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'data'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,2022-07-20 17:30:57,313.1,311.1,1016,15,281.58,0,0.89,262


## Function hist_weather(unix_datetime)

In [6]:
def hist_weather(unix_datetime, lat, lon):
    '''
    Using the OpenWeather OneCall 3.0 API to pull historical daily data
    Input: UNIX datetime for day requesting
    Output: Pandas dataframe for that day's weather
    '''
    
    url_hist_point = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_datetime}&appid=d43015e86fe9a04995afe46ef9ad42c4"
    hist_point_req = requests.get(url_hist_point)
    print(f'API Status: {hist_point_req.status_code}')

    wd_hist = hist_point_req.json()

    wd_hist_df = json_normalize(wd_hist['data'])
    wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
           'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

    # Convert dt UNIX to datetime UTC
    wd_hist_df['datetime'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_hist_df

# Testing: Current/7-Day Forecast Daily Weather from One Call API

In [7]:
url_current_daily = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,hourly,alerts&appid=d43015e86fe9a04995afe46ef9ad42c4"
current_daily_req = requests.get(url_current_daily)
print(f'Status: {current_daily_req.status_code}')

wd_current = current_daily_req.json()
print(wd_current.keys())

wd_current_df = json_normalize(wd_current['daily'])
wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# TEMPERATURE - KELVINS
temp_list = []
for i in range(len(wd_current['daily'])):
    avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
    temp_list.append(avg_temp)
wd_current_df['temp_avg'] = temp_list

# FEELS LIKE - KELVINS
fl_list = []
for i in range(len(wd_current['daily'])):
    avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
    fl_list.append(avg_fl)
wd_current_df['feels_like_avg'] = fl_list

# Convert dt UNIX to datetime UTC
wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_current_df.head(10)

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'daily'])


,dt,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,wind_gust,temp_avg,feels_like_avg,datetime
0,1660075200,1013,23,280.45,4,5.63,215,6.60,298.40,296.96,2022-08-09 13:00:00
1,1660161600,1016,22,279.68,0,4.05,214,6.60,298.30,297.13,2022-08-10 13:00:00
2,1660248000,1014,15,276.71,5,3.87,198,7.09,300.73,299.14,2022-08-11 13:00:00
3,1660334400,1012,12,275.27,0,3.68,186,6.47,301.20,299.50,2022-08-12 13:00:00
4,1660420800,1012,9,272.21,0,3.54,195,6.58,302.13,300.43,2022-08-13 13:00:00
5,1660507200,1010,10,272.94,2,4.06,178,6.61,300.93,299.10,2022-08-14 13:00:00
6,1660593600,1008,14,277.34,0,3.36,184,5.97,300.86,299.32,2022-08-15 13:00:00
7,1660680000,1008,11,276.04,0,2.76,147,5.08,303.83,302.29,2022-08-16 13:00:00


## Function curr_weather(lat, lon):

In [8]:
def curr_weather(lat, lon):
    '''
    Using OpenWeather OneCall 3.0 API, takes in the latitude and longitude of a location and returns a dataframe with current + 7-day forecast.
    Input: latitude and longitude of a location
    Output: dataframe
    '''
    
    url_current_daily = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid=d43015e86fe9a04995afe46ef9ad42c4"
    current_daily_req = requests.get(url_current_daily)
    print(f'Status: {current_daily_req.status_code}')

    wd_current = current_daily_req.json()
    print(wd_current.keys())

    wd_current_df = json_normalize(wd_current['daily'])
    wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
           'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

    # TEMPERATURE - KELVINS
    temp_list = []
    for i in range(len(wd_current['daily'])):
        avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
        temp_list.append(avg_temp)
    wd_current_df['temp_avg'] = temp_list

    # FEELS LIKE - KELVINS
    fl_list = []
    for i in range(len(wd_current['daily'])):
        avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
        fl_list.append(avg_fl)
    wd_current_df['feels_like_avg'] = fl_list

    # Convert dt UNIX to datetime UTC
    wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_current_df

# Hourly - Need Paid Subscription for Historical Hourly

## Testing: Current/2-Day Forecast Hourly Weather from One Call API
---

In [9]:
url_current_hourly = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,daily&appid=d43015e86fe9a04995afe46ef9ad42c4"
current_hourly_req = requests.get(url_current_hourly)
print(f'Status: {current_hourly_req.status_code}')

wh_current = current_hourly_req.json()
print(wh_current.keys())

wh_current_df = json_normalize(wh_current['hourly'])
wh_current_df = wh_current_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# Convert dt UNIX to datetime UTC
wh_current_df['datetime'] = wh_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wh_current_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'hourly'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,wind_gust,datetime
0,1660050000,291.17,290.88,1012,71,285.84,7,3.16,152,5.44,2022-08-09 06:00:00
1,1660053600,291.32,291.07,1012,72,286.20,0,3.07,149,5.26,2022-08-09 07:00:00
2,1660057200,291.54,291.26,1012,70,285.98,3,3.21,154,4.55,2022-08-09 08:00:00
3,1660060800,292.68,292.36,1013,64,285.70,5,3.19,166,4.00,2022-08-09 09:00:00
4,1660064400,294.78,294.43,1013,55,285.36,6,2.95,179,3.44,2022-08-09 10:00:00


## Testing: Historical Hourly Weather from History API

In [10]:
'''
url_hist_hourly = "http://history.openweathermap.org/data/2.5/history/city?lat=38.752125&lon=-121.288010&type=hour&start=1659470400&end=1659470400&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_hourly_req = requests.get(url_hist_hourly)
print(f'Status: {hist_hourly_req.status_code}')

wh_hist = hist_hourly_req.json()
print(wh_hist.keys())

wh_hist_df = json_normalize(wh_hist['data'])
wh_hist_df = wh_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg',
       'wind_gust']]
wh_hist_df.head()
'''

'\nurl_hist_hourly = "http://history.openweathermap.org/data/2.5/history/city?lat=38.752125&lon=-121.288010&type=hour&start=1659470400&end=1659470400&appid=d43015e86fe9a04995afe46ef9ad42c4"\nhist_hourly_req = requests.get(url_hist_hourly)\nprint(f\'Status: {hist_hourly_req.status_code}\')\n\nwh_hist = hist_hourly_req.json()\nprint(wh_hist.keys())\n\nwh_hist_df = json_normalize(wh_hist[\'data\'])\nwh_hist_df = wh_hist_df[[\'dt\', \'temp\', \'feels_like\', \'pressure\', \'humidity\',\n       \'dew_point\', \'clouds\', \'wind_speed\', \'wind_deg\',\n       \'wind_gust\']]\nwh_hist_df.head()\n'